#Aula 11 - Supercomputação 24.2
###Alexandre Santarossa

Cálculo da Variância usando Iteradores Dinâmicos

In [44]:
%%writefile variancia.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform_reduce.h>
#include <iostream>
#include <fstream>
#include <cmath>

struct variancia_op {
    double media;
    variancia_op(double _media) : media(_media) {}

    __host__ __device__
    double operator()(const double& x) const {
        double diff = x - media;
        return diff * diff;
    }
};

int main() {
    std::ifstream file("stocks-google.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt" << std::endl;
        return 1;
    }

    thrust::host_vector<double> host_stocks;
    double value;
    while (file >> value) {
        host_stocks.push_back(value);
    }
    file.close();

    if (host_stocks.empty()) {
        std::cerr << "O arquivo está vazio ou não contém dados válidos." << std::endl;
        return 1;
    }

    thrust::device_vector<double> stocks = host_stocks;
    size_t n = stocks.size();

    double soma = thrust::reduce(stocks.begin(), stocks.end(), 0.0, thrust::plus<double>());
    double media = soma / static_cast<double>(n);

    variancia_op op(media);
    double variancia = thrust::transform_reduce(
        stocks.begin(),
        stocks.end(),
        op,
        0.0,
        thrust::plus<double>()
    );

    variancia /= static_cast<double>(n);

    std::cout << "Variância: " << variancia << std::endl;
    return 0;
}

Overwriting variancia.cu


In [45]:
!nvcc -arch=sm_75 -std=c++14 variancia.cu -o variancia

In [46]:
!./variancia

Variância: 123792


Análise de Variação Diária dos Preços de Ações

In [33]:
%%writefile diferenca_diaria.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <iostream>
#include <fstream>

struct diferenca_diaria {
    __host__ __device__
    double operator()(const double& next, const double& prev) const {
        return next - prev;
    }
};

int main() {
    std::ifstream file("stocks-google.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt" << std::endl;
        return 1;
    }

    thrust::host_vector<double> host_stocks;
    double value;
    while (file >> value) {
        host_stocks.push_back(value);
    }
    file.close();

    if (host_stocks.size() < 2) {
        std::cerr << "O arquivo deve conter pelo menos dois valores para calcular a diferença diária." << std::endl;
        return 1;
    }

    thrust::device_vector<double> stocks = host_stocks;
    size_t n = stocks.size();
    thrust::device_vector<double> ganho_diario(n - 1);

    thrust::transform(
        stocks.begin() + 1,
        stocks.end(),
        stocks.begin(),
        ganho_diario.begin(),
        diferenca_diaria()
    );

    std::cout << "Ganho Diário: ";
    for (size_t i = 0; i < ganho_diario.size(); ++i) {
        std::cout << ganho_diario[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}

Overwriting diferenca_diaria.cu


In [47]:
!nvcc -arch=sm_75 -std=c++14 diferenca_diaria.cu -o diferenca_diaria

In [48]:
!./diferenca_diaria

Ganho Diário: 5.26 1.5 -3.25 6.25 3.5 3.25 -1.75 5 -2.5 5.5 -5.25 6.25 1.5 2.5 2.25 -5.25 -1.5 3.75 5.5 1.25 2.5 2.5 4.25 2.25 5.25 2.25 2.5 2.75 2.5 2.25 2.25 2.25 3.5 2.25 2.5 2.25 3.5 2.25 2.25 4.5 3.25 1.25 3.25 3.25 2.25 3.25 2.5 2.5 3.25 2.5 2.75 2.5 2.25 2.75 2.5 2.5 2.25 3.5 2.25 2.5 2.25 3.5 3.25 2.5 3.25 2.75 2.75 3.25 3.25 2.75 3.5 2.5 2.5 2.25 3.25 2.25 4.25 2.5 3.75 3.25 2.25 3.25 2.5 3.25 3.25 2.5 3.75 2.25 2.5 2.5 3.25 2.25 3.5 3.25 2.75 3.75 3.75 2.5 3.25 3.5 2.25 3.5 3.25 3.5 2.75 3.5 3.5 2.5 3.25 3.25 3.5 2.5 3.25 2.5 3.25 3.5 2.5 3.25 2.25 3.25 3.25 3.25 2.75 2.5 3.5 2.75 2.75 2.75 2.75 3.75 3.25 3.5 2.5 3.25 3.5 2.75 2.5 3.25 3.25 2.5 3.75 2.25 3.5 3.25 3.5 2.75 3.5 3.5 3.25 2.25 3.25 3.25 3.25 3.25 2.5 3.75 3.25 3.5 3.5 2.5 3.25 3.25 3.25 3.25 2.5 3.25 3.5 3.25 3.5 3.25 2.75 3.5 3.25 3.25 3.25 3.25 2.5 3.25 2.25 3.5 3.5 3.25 3.25 3.5 3.25 2.75 3.75 3.25 3.5 3.25 2.75 3.5 3.25 3.5 2.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.5 3.25 3.

Contagem de Dias com Aumento no Preço das Ações

In [49]:
%%writefile dias_com_aumento.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/count.h>
#include <thrust/transform.h>
#include <iostream>
#include <fstream>

struct is_positive {
    __host__ __device__
    bool operator()(const double& x) const {
        return x > 0;
    }
};

struct diferenca_diaria {
    __host__ __device__
    double operator()(const double& next, const double& prev) const {
        return next - prev;
    }
};

int main() {
    std::ifstream file("stocks-google.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt" << std::endl;
        return 1;
    }

    thrust::host_vector<double> host_stocks;
    double value;
    while (file >> value) {
        host_stocks.push_back(value);
    }
    file.close();

    if (host_stocks.size() < 2) {
        std::cerr << "O arquivo deve conter pelo menos dois valores para calcular a diferença diária." << std::endl;
        return 1;
    }

    thrust::device_vector<double> stocks = host_stocks;
    size_t n = stocks.size();
    thrust::device_vector<double> ganho_diario(n - 1);

    thrust::transform(
        stocks.begin() + 1,
        stocks.end(),
        stocks.begin(),
        ganho_diario.begin(),
        diferenca_diaria()
    );

    size_t dias_com_aumento = thrust::count_if(
        ganho_diario.begin(),
        ganho_diario.end(),
        is_positive()
    );

    std::cout << "Dias com aumento no preço: " << dias_com_aumento << std::endl;
    return 0;
}

Overwriting dias_com_aumento.cu


In [50]:
!nvcc -arch=sm_75 -std=c++14 dias_com_aumento.cu -o dias_com_aumento

In [51]:
!./dias_com_aumento

Dias com aumento no preço: 3056


Cálculo do Aumento Médio nos Dias em que o Preço Subiu

In [52]:
%%writefile aumento_medio.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/replace.h>
#include <thrust/reduce.h>
#include <thrust/count.h>
#include <iostream>
#include <fstream>

struct is_negative {
    __host__ __device__
    bool operator()(const double& x) const {
        return x < 0;
    }
};

struct is_positive {
    __host__ __device__
    bool operator()(const double& x) const {
        return x > 0;
    }
};

struct diferenca_diaria {
    __host__ __device__
    double operator()(const double& next, const double& prev) const {
        return next - prev;
    }
};

int main() {
    std::ifstream file("stocks-google.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt" << std::endl;
        return 1;
    }

    thrust::host_vector<double> host_stocks;
    double value;
    while (file >> value) {
        host_stocks.push_back(value);
    }
    file.close();

    if (host_stocks.size() < 2) {
        std::cerr << "O arquivo deve conter pelo menos dois valores para calcular a diferença diária." << std::endl;
        return 1;
    }

    thrust::device_vector<double> stocks = host_stocks;
    size_t n = stocks.size();
    thrust::device_vector<double> ganho_diario(n - 1);

    thrust::transform(
        stocks.begin() + 1,
        stocks.end(),
        stocks.begin(),
        ganho_diario.begin(),
        diferenca_diaria()
    );

    thrust::replace_if(
        ganho_diario.begin(),
        ganho_diario.end(),
        is_negative(),
        0.0
    );

    double soma_aumentos = thrust::reduce(
        ganho_diario.begin(),
        ganho_diario.end(),
        0.0,
        thrust::plus<double>()
    );

    size_t dias_com_aumento = thrust::count_if(
        ganho_diario.begin(),
        ganho_diario.end(),
        is_positive()
    );

    if (dias_com_aumento == 0) {
        std::cerr << "Nenhum dia com aumento encontrado. Não é possível calcular a média." << std::endl;
        return 1;
    }

    double aumento_medio = soma_aumentos / static_cast<double>(dias_com_aumento);

    std::cout << "Aumento médio nos dias com aumento: " << aumento_medio << std::endl;

    return 0;
}

Overwriting aumento_medio.cu


In [53]:
!nvcc -arch=sm_75 -std=c++14 aumento_medio.cu -o aumento_medio

In [54]:
!./aumento_medio

Aumento médio nos dias com aumento: 3.15251
